In [2]:
# Our essentials
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier



import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from data_clean import confusion_table, replace_obj_cols
#Acquire functions
from acquire import get_connection, new_telco_churn_data, get_telco_churn_data


In [3]:
df = get_telco_churn_data()

In [4]:
df

,payment_type_id,contract_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,contract_type,payment_type
0,2,1,3,0030-FNXPP,Female,0,No,No,3,Yes,...,No internet service,No internet service,No internet service,No,19.85,57.2,No,None,Month-to-month,Mailed check
1,2,1,3,0031-PVLZI,Female,0,Yes,Yes,4,Yes,...,No internet service,No internet service,No internet service,No,20.35,76.35,Yes,None,Month-to-month,Mailed check
2,1,1,3,0098-BOWSO,Male,0,No,No,27,Yes,...,No internet service,No internet service,No internet service,Yes,19.40,529.8,No,None,Month-to-month,Electronic check
3,1,1,3,0107-WESLM,Male,0,No,No,1,Yes,...,No internet service,No internet service,No internet service,Yes,19.85,19.85,Yes,None,Month-to-month,Electronic check
4,3,1,3,0114-RSRRW,Female,0,Yes,No,10,Yes,...,No internet service,No internet service,No internet service,Yes,19.95,187.75,No,None,Month-to-month,Bank transfer (automatic)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,4,3,1,9919-FZDED,Male,1,Yes,No,72,Yes,...,No,Yes,Yes,Yes,84.10,5981.65,No,DSL,Two year,Credit card (automatic)
7039,3,3,1,9926-PJHDQ,Female,0,Yes,Yes,72,Yes,...,No,Yes,Yes,Yes,76.80,5468.45,No,DSL,Two year,Bank transfer (automatic)
7040,2,3,1,9928-BZVLZ,Female,0,No,No,12,No,...,Yes,No,Yes,No,49.85,552.1,No,DSL,Two year,Mailed check
7041,2,3,1,9993-LHIEB,Male,0,Yes,Yes,67,Yes,...,Yes,No,Yes,No,67.85,4627.65,No,DSL,Two year,Mailed check


In [5]:
#2.) Drop the customer_id columns.
df = df.drop(columns =['customer_id'])

In [6]:
# information about the dataframe: column names, rows, datatypes, non-missing values

from sklearn.model_selection import train_test_split

def train_validate_test_split(df, target, seed=123):
    '''
    This function takes in a dataframe, the name of the target variable
    (for stratification purposes), and an integer for a setting a seed
    and splits the data into train, validate and test. 
    Test is 20% of the original dataset, validate is .30*.80= 24% of the 
    original dataset, and train is .70*.80= 56% of the original dataset. 
    The function returns, in this order, train, validate and test dataframes. 
    '''
    train_validate, test = train_test_split(df, test_size=0.2, 
                                            random_state=seed, 
                                            stratify=df[target])
    train, validate = train_test_split(train_validate, test_size=0.3, 
                                       random_state=seed,
                                       stratify=train_validate[target])
    return train, validate, test


df['monthly_charges'] = pd.to_numeric(df.monthly_charges, errors = 'coerce', downcast = 'float')
df.imputer = SimpleImputer(strategy = 'mean')
df.imputer = SimpleImputer(strategy = 'most_frequent')
    
#
df['monthly_charges'] = df.imputer.fit_transform(df[['monthly_charges']])
df['total_charges'] = df.imputer.fit_transform(df[['total_charges']])

                                  
                                  
                                  
df, replace_key, revert_key = replace_obj_cols(df)
                                  
df_train, df_validate, df_test = train_validate_test_split(df, target = 'churn')
                                
df_train['baseline'] = False
ipt_df_t = df_train[['churn', 'baseline']]
ipt_df_v = df_validate[['churn']]                                 
ipt_df_test = df_test[['churn']]
                                  
                                  
df_train.drop(columns = 'baseline', inplace = True)                                  
 
clf = DecisionTreeClassifier(max_depth = 24, splitter = 'best')   

                                  

In [7]:
# Create X & y version of train, validate, test where y is just the
# Series with the target variable and X is all of the features

X_df_train= df_train.drop(columns=['churn'])
y_df_train = df_train['churn']



X_df_validate = df_validate.drop(columns=['churn'])
y_df_validate = df_validate['churn']

X_df_test = df_test.drop(columns=['churn'])
y_df_test = df_test['churn']




print(X_df_train.shape, X_df_validate.shape, X_df_test.shape)

clf_df = DecisionTreeClassifier(max_depth=5, splitter='best')
clf_df = clf_df.fit(X_df_train, y_df_train)


y_pred_df = clf_df.predict(X_df_train)
y_pred_proba_df = clf_df.predict_proba(X_df_train)


print(f'Accuracy of Decision Tree classifier on training set: {clf_df.score(X_df_validate, y_df_validate)}')

y_pred1_tdf = clf_df.predict(X_df_validate)
ipt_df_v['y_pred1_max_depth_5'] = clf_df.predict(X_df_validate)
print(confusion_table(ipt_df_v))

(3943, 22) (1691, 22) (1409, 22)
Accuracy of Decision Tree classifier on training set: 0.7717327025428741


ImportError: Missing optional dependency 'tabulate'.  Use pip or conda to install tabulate.

In [22]:
df = get_telco_churn_data()

In [23]:
df

,payment_type_id,contract_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,contract_type,payment_type
0,2,1,3,0030-FNXPP,Female,0,No,No,3,Yes,...,No internet service,No internet service,No internet service,No,19.85,57.2,No,None,Month-to-month,Mailed check
1,2,1,3,0031-PVLZI,Female,0,Yes,Yes,4,Yes,...,No internet service,No internet service,No internet service,No,20.35,76.35,Yes,None,Month-to-month,Mailed check
2,1,1,3,0098-BOWSO,Male,0,No,No,27,Yes,...,No internet service,No internet service,No internet service,Yes,19.40,529.8,No,None,Month-to-month,Electronic check
3,1,1,3,0107-WESLM,Male,0,No,No,1,Yes,...,No internet service,No internet service,No internet service,Yes,19.85,19.85,Yes,None,Month-to-month,Electronic check
4,3,1,3,0114-RSRRW,Female,0,Yes,No,10,Yes,...,No internet service,No internet service,No internet service,Yes,19.95,187.75,No,None,Month-to-month,Bank transfer (automatic)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,4,3,1,9919-FZDED,Male,1,Yes,No,72,Yes,...,No,Yes,Yes,Yes,84.10,5981.65,No,DSL,Two year,Credit card (automatic)
7039,3,3,1,9926-PJHDQ,Female,0,Yes,Yes,72,Yes,...,No,Yes,Yes,Yes,76.80,5468.45,No,DSL,Two year,Bank transfer (automatic)
7040,2,3,1,9928-BZVLZ,Female,0,No,No,12,No,...,Yes,No,Yes,No,49.85,552.1,No,DSL,Two year,Mailed check
7041,2,3,1,9993-LHIEB,Male,0,Yes,Yes,67,Yes,...,Yes,No,Yes,No,67.85,4627.65,No,DSL,Two year,Mailed check


In [24]:
#2.) Drop the customer_id columns.
df = df.drop(columns =['customer_id'])

In [32]:
# information about the dataframe: column names, rows, datatypes, non-missing values

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=1,
                            n_estimators=100,
                            max_depth=10, 
                            random_state=123)

from sklearn.model_selection import train_test_split

def train_validate_test_split(df, target, seed=123):
    '''
    This function takes in a dataframe, the name of the target variable
    (for stratification purposes), and an integer for a setting a seed
    and splits the data into train, validate and test. 
    Test is 20% of the original dataset, validate is .30*.80= 24% of the 
    original dataset, and train is .70*.80= 56% of the original dataset. 
    The function returns, in this order, train, validate and test dataframes. 
    '''
    train_validate, test = train_test_split(df, test_size=0.2, 
                                            random_state=seed, 
                                            stratify=df[target])
    train, validate = train_test_split(train_validate, test_size=0.3, 
                                       random_state=seed,
                                       stratify=train_validate[target])
    return train, validate, test


df['monthly_charges'] = pd.to_numeric(df.monthly_charges, errors = 'coerce', downcast = 'float')
df.imputer = SimpleImputer(strategy = 'mean')
df.imputer = SimpleImputer(strategy = 'most_frequent')
    
#
df['monthly_charges'] = df.imputer.fit_transform(df[['monthly_charges']])
df['total_charges'] = df.imputer.fit_transform(df[['total_charges']])

                                  
                                  
                                  
df, replace_key, revert_key = replace_obj_cols(df)
                                  
df_train, df_validate, df_test = train_validate_test_split(df, target = 'churn')
                                
df_train['baseline'] = False
ipt_df_t = df_train[['churn', 'baseline']]
ipt_df_v = df_validate[['churn']]                                 
ipt_df_test = df_test[['churn']]
                                  
                                  
df_train.drop(columns = 'baseline', inplace = True)                                  
 
rlf = RandomForestClassifier(max_depth = 10, random_state = 123)   


In [33]:
# Create X & y version of train, validate, test where y is just the
# Series with the target variable and X is all of the features

X_df_train= df_train.drop(columns=['churn'])
y_df_train = df_train['churn']



X_df_validate = df_validate.drop(columns=['churn'])
y_df_validate = df_validate['churn']

X_df_test = df_test.drop(columns=['churn'])
y_df_test = df_test['churn']




print(X_df_train.shape, X_df_validate.shape, X_df_test.shape)

rlf_df = RandomForestClassifier(max_depth=10)
rlf_df = clf_df.fit(X_df_train, y_df_train)


y_pred_df = rlf_df.predict(X_df_train)
y_pred_proba_df = rlf_df.predict_proba(X_df_train)


print(f'Accuracy of RandomForestClassifier on training set: {rlf_df.score(X_df_validate, y_df_validate)}')

y_pred1_tdf = rlf_df.predict(X_df_validate)
ipt_df_v['y_pred1_max_depth_5'] = rlf_df.predict(X_df_validate)
print(confusion_table(ipt_df_v))

(3943, 22) (1691, 22) (1409, 22)
Accuracy of RandomForestClassifier on training set: 0.7705499704316973


ImportError: Missing optional dependency 'tabulate'.  Use pip or conda to install tabulate.

In [34]:
rf.fit(df_train, y_df_train)

RandomForestClassifier(max_depth=10, random_state=123)

In [35]:
print(rf.feature_importances_)

[0.0212222  0.02230785 0.00572766 0.0026102  0.00303864 0.00261665
 0.00275054 0.05001101 0.0008532  0.00317836 0.00593169 0.00639749
 0.00849148 0.00726946 0.00429292 0.00485019 0.00346394 0.02755216
 0.05615273 0.70986063 0.00961604 0.03561511 0.00618983]


In [36]:
y_pred = rf.predict(X_df_train)

ValueError: X has 22 features, but DecisionTreeClassifier is expecting 23 features as input.